[Reference](https://leapcell.medium.com/peewee-orm-the-most-elegant-python-orm-ever-4bc0167714fe)

# (1) Creating Records

In [1]:
!pip install peewee

In [2]:
from peewee import *

In [3]:
# Using SQLite for demonstration; replace with actual database in Leapcell deployment
db = SqliteDatabase('leapcell_users.db')
class User(Model):
    username = CharField()
    email = CharField(unique=True)
    plan_type = CharField()
    class Meta:
        database = db
db.connect()
db.create_tables([User])
# Create a new user using Model.create()
new_user = User.create(username='test_user', email='test@example.com', plan_type='basic')

# (2) Batch Insertion

In [4]:
user_data = [
    {'username': 'user1', 'email': 'user1@example.com', 'plan_type': 'pro'},
    {'username': 'user2', 'email': 'user2@example.com', 'plan_type': 'basic'},
    {'username': 'user3', 'email': 'user3@example.com', 'plan_type': 'enterprise'}
]

In [5]:
# Batch insert using insert_many()
with db.atomic():
    User.insert_many(user_data).execute()

# (3) Updating Records

In [6]:
# Update a single user
user_to_update = User.get(User.username == 'test_user')
user_to_update.plan_type = 'pro'
user_to_update.save()

1

In [7]:
# Batch update: upgrade all basic users to pro
query = User.update(plan_type='pro').where(User.plan_type == 'basic')
query.execute()

1

# (4) Deleting Records

In [8]:
# Delete a single user
user_to_delete = User.get(User.username == 'user1')
user_to_delete.delete_instance()

1

In [10]:
# # Delete multiple users (e.g., inactive accounts)
# query = User.delete().where(User.is_deleted == True)
# query.execute()

# (5) Querying Records

In [11]:
# Query by primary key
user = User.get_by_id(1)
print(user.username, user.email, user.plan_type)

test_user test@example.com pro


In [12]:
# Query by other fields
user = User.get(User.email == 'test@example.com')

In [13]:
print(user)

1


In [14]:
# Iterate over all users
for user in User.select():
    print(user.username, user.email)

test_user test@example.com
user2 user2@example.com
user3 user3@example.com


In [15]:
# Slicing and indexing
users_subset = User.select()[:5]
for user in users_subset:
    print(user.username)

test_user
user2
user3


In [16]:
# Simple filter
pro_users = User.select().where(User.plan_type == 'pro')

In [18]:
# Ascending order by username
sorted_users = User.select().order_by(User.username)

In [20]:
# Paginate results (page 2, 10 records per page)
paged_users = User.select().order_by(User.id).paginate(2, 10)
for user in paged_users:
    print(user.username)

In [21]:
# Count total users
user_count = User.select().count()
print(f"Total users: {user_count}")
# Count users by plan type
pro_user_count = User.select().where(User.plan_type == 'pro').count()

Total users: 3


In [22]:
from peewee import fn

# Count users per plan type
query = (User
        .select(User.plan_type, fn.Count(User.id).alias('count'))
        .group_by(User.plan_type))
for result in query:
    print(result.plan_type, result.count)
# Retrieve a scalar value (e.g., max user ID)
max_id = User.select(fn.Max(User.id)).scalar()

enterprise 1
pro 2
